In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import re  #正则表达式
import nltk 
from nltk.corpus import stopwordsci

ModuleNotFoundError: No module named 'nltk'

## 删除html页面标记

In [3]:
#删除html页面标记
text = '''<p><font> Big data refers to the large, diverse sets of information that grow at ever-increasing rates. 
        It encompasses the volume of information, the velocity or speed at which it is created and collected, 
        and the variety or scope of the data points being covered. 
        Big data often comes from multiple sources and arrives in multiple formats.</p><br//>'''

#compile函数生成正则表达式pattern，匹配html标签<>. re.S 匹配包括换行在内的所有字符
pattern = re.compile(r'<[^>]+>',re.S) 
result = pattern.sub('', text) #用空字符替换掉匹配的html标记

result.lower()  #英文转换小写
print(result)

 Big data refers to the large, diverse sets of information that grow at ever-increasing rates. 
        It encompasses the volume of information, the velocity or speed at which it is created and collected, 
        and the variety or scope of the data points being covered. 
        Big data often comes from multiple sources and arrives in multiple formats.


## 去除标点及停用词 

In [4]:
result=re.sub(r'[^\w\s]','',result)  #正则表达式，去除标点符号

tokenize = nltk.word_tokenize(result) #英文分词

tokenize_new = [w for w in tokenize if not w in stopwords.words('english')] #去除停用词
tokenize_new

NameError: name 'nltk' is not defined

## 词干提取 

In [5]:
#词干提取
s = nltk.stem.SnowballStemmer('english')  
stemmer = [s.stem(ws) for ws in tokenize_new]
print(stemmer)


NameError: name 'nltk' is not defined

## 词性标注

In [5]:
#词性标注
pos_tags=nltk.pos_tag(tokenize_new)#词性标注
print(pos_tags)
tags = set(['MD', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'RP', 'RB', 'RBR', 'RBS', 'JJ', 'JJR', 'JJS'])#根据词性选择
ret = []
for word,pos in pos_tags:
        if (pos in tags):
            ret.append(word)
newdata= ' '.join(ret)
print(newdata)



[('Big', 'NNP'), ('data', 'NN'), ('refers', 'NNS'), ('large', 'JJ'), ('diverse', 'JJ'), ('sets', 'NNS'), ('information', 'NN'), ('grow', 'NN'), ('everincreasing', 'VBG'), ('rates', 'NNS'), ('It', 'PRP'), ('encompasses', 'VBZ'), ('volume', 'NN'), ('information', 'NN'), ('velocity', 'NN'), ('speed', 'NN'), ('created', 'VBD'), ('collected', 'JJ'), ('variety', 'NN'), ('scope', 'NN'), ('data', 'NNS'), ('points', 'NNS'), ('covered', 'VBD'), ('Big', 'NNP'), ('data', 'NNS'), ('often', 'RB'), ('comes', 'VBZ'), ('multiple', 'JJ'), ('sources', 'NNS'), ('arrives', 'VBZ'), ('multiple', 'JJ'), ('formats', 'NNS')]
large diverse everincreasing encompasses created collected covered often comes multiple arrives multiple


##  词形还原

In [6]:
from nltk.corpus import wordnet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
wnl = nltk.stem.WordNetLemmatizer()
words_lemma=[]
for pos in pos_tags:  #pos_tags 上一步词性标注结果
    if get_wordnet_pos(pos[1])!=None : #根据上面函数，J,V,N,R开头的保留，其他返回空值，如果不为空，表示如果为JVNR
        wordnet_pos =get_wordnet_pos(pos[1]) #pos[1]为标注的词性
        words_lemma.append(wnl.lemmatize(pos[0], pos=wordnet_pos)) # 词形还原     
print(words_lemma)

['Big', 'data', 'refers', 'large', 'diverse', 'set', 'information', 'grow', 'everincreasing', 'rate', 'encompass', 'volume', 'information', 'velocity', 'speed', 'create', 'collected', 'variety', 'scope', 'data', 'point', 'cover', 'Big', 'data', 'often', 'come', 'multiple', 'source', 'arrive', 'multiple', 'format']


## 词频统计 

In [7]:
#词频统计
from nltk import FreqDist    
freq = FreqDist(words_lemma)    #words_lemma:上一步词形还原结果
for key,val in freq.items():
    print (str(key) + ':' + str(val))

Big:2
data:3
refers:1
large:1
diverse:1
set:1
information:2
grow:1
everincreasing:1
rate:1
encompass:1
volume:1
velocity:1
speed:1
create:1
collected:1
variety:1
scope:1
point:1
cover:1
often:1
come:1
multiple:2
source:1
arrive:1
format:1


## 词云 

In [1]:
#词云
from wordcloud import WordCloud
words = ' '.join(words_lemma)
wc = WordCloud(background_color = 'white',width=800, height=600)
wwc = wc.generate(words)
plt.figure(figsize=(10,10))
plt.imshow(wwc)
plt.axis("off")
plt.show()

NameError: name 'words_lemma' is not defined

In [9]:
#中文分词
import jieba 
sent = '南京市长江大桥'  
seg_list = jieba.cut(sent)    #jieba分词默认模式 
print('分词结果', '/ '.join(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Yjh\AppData\Local\Temp\jieba.cache
Loading model cost 0.763 seconds.
Prefix dict has been built successfully.


分词结果 南京市/ 长江大桥


In [10]:
import jieba.posseg as psg 

sent = '中文分词是文本处理不可或缺的一步，地球人加油！'  
seg_list = psg.cut(sent)  # Jieba词性标注
print(' '.join(['{0}/{1}'.format(w, t) for w, t in seg_list]))


中文/nz 分词/n 是/v 文本处理/n 不可或缺/l 的/uj 一步/m ，/x 地球/n 人/n 加油/v ！/x


## 文本表示 

### 词袋子 

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['This is the first document.',
          'This document is the second document.',
          'And this is the third one.',
          'Is this the first document?']

vectorizer = CountVectorizer(ngram_range=(1, 2)) #创建词袋子  1元 和2元，默认1元
dt = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(dt.toarray())  

['and' 'and this' 'document' 'document is' 'first' 'first document' 'is'
 'is the' 'is this' 'one' 'second' 'second document' 'the' 'the first'
 'the second' 'the third' 'third' 'third one' 'this' 'this document'
 'this is' 'this the']
[[0 0 1 0 1 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 1 0]
 [0 0 2 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 1 1 0 0]
 [1 1 0 0 0 0 1 1 0 1 0 0 1 0 0 1 1 1 1 0 1 0]
 [0 0 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 0 1 0 0 1]]


### TFIDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2))#参数默认1元
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(X.toarray())

['and' 'and this' 'document' 'document is' 'first' 'first document' 'is'
 'is the' 'is this' 'one' 'second' 'second document' 'the' 'the first'
 'the second' 'the third' 'third' 'third one' 'this' 'this document'
 'this is' 'this the']
[[0.         0.         0.3145322  0.         0.38850984 0.38850984
  0.25715068 0.3145322  0.         0.         0.         0.
  0.25715068 0.38850984 0.         0.         0.         0.
  0.25715068 0.         0.38850984 0.        ]
 [0.         0.         0.45551258 0.35682424 0.         0.
  0.18620569 0.22775629 0.         0.         0.35682424 0.35682424
  0.18620569 0.         0.35682424 0.         0.         0.
  0.18620569 0.35682424 0.         0.        ]
 [0.35700721 0.35700721 0.         0.         0.         0.
  0.18630117 0.22787308 0.         0.35700721 0.         0.
  0.18630117 0.         0.         0.35700721 0.35700721 0.35700721
  0.18630117 0.         0.28146859 0.        ]
 [0.         0.         0.28293955 0.         0.34948664 0.

### LSA

In [13]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(3)
X1=svd.fit_transform(X)
print(X1)

[[ 0.89256849 -0.09143315 -0.18907985]
 [ 0.59110831 -0.04894607  0.8032774 ]
 [ 0.47733484  0.86478807 -0.11173105]
 [ 0.80235771 -0.37670319 -0.31497686]]
